# **Data Cleaning**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#Import necessary Libraries
import pandas as pd
import numpy as np

In [3]:
path = "drive/MyDrive/Colab Notebooks/"

In [4]:
#Raw data received from Web Scraping
RawData = pd.read_csv(path + "transfermarkt_data_with_competitions_and_weather_complete.csv")

In [5]:
RawData.head()

,Competition,Matchday,Date,Time,Home Team,Ranking Home Team,Away Team,Ranking Away Team,Attendance,Result,Weather,Temperature (°C)
0,Super League,1,Sun 7/21/19,4:00 PM,FC Zürich,(7.),FC Lugano,(5.),9.430,0:4,Clear or mostly clear,26.8
1,Super League,2,Sun 7/28/19,4:00 PM,FC Luzern,(3.),FC Zürich,(10.),9.917,0:0,Rainy,17.7
2,Super League,3,Sat 8/3/19,7:00 PM,FC Sion,(9.),FC Zürich,(10.),9.800,3:1,Partly cloudy,23.0
3,Super League,4,Sun 8/11/19,4:00 PM,FC Zürich,(10.),Neuchâtel Xamax,(9.),9.619,2:2,Partly cloudy,26.6
4,Super League,6,Wed 8/14/19,7:00 PM,FC Zürich,(10.),FC St. Gallen,(5.),9.112,2:1,Clear or mostly clear,20.4


In [6]:
RawData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2866 entries, 0 to 2865
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Competition        2866 non-null   object 
 1   Matchday           2866 non-null   object 
 2   Date               2866 non-null   object 
 3   Time               2866 non-null   object 
 4   Home Team          2866 non-null   object 
 5   Ranking Home Team  2365 non-null   object 
 6   Away Team          2866 non-null   object 
 7   Ranking Away Team  2410 non-null   object 
 8   Attendance         2774 non-null   object 
 9   Result             2866 non-null   object 
 10  Weather            2302 non-null   object 
 11  Temperature (°C)   2302 non-null   float64
dtypes: float64(1), object(11)
memory usage: 268.8+ KB


## Date Handling

The date format must first be transformed. To do this, the day of the week, month and year are extracted from the date column ans displayed as separate columns:

- column date (e.g. 7/21/2019)
- column weekday (e.g. Sunday)
- column month (e.g.7)
- year (e.g. 2019)

In [7]:
RawData = RawData.rename(columns={'Date': 'old_date'})

In [8]:
#Change date format
RawData['date'] = pd.to_datetime(RawData['old_date'], format='%a %m/%d/%y')
# Changed the format to '%a %m/%d/%y' to include the weekday abbreviation
# '%a' represents the abbreviated weekday name (e.g., Sun, Mon, Tue)

# Extracting & creating new columns for date, Weekday, Month and Year from the Date column
RawData['Date'] = RawData['date'].dt.strftime('%m-%d-%Y')
RawData['Weekday'] = RawData['date'].dt.day_name()
RawData['Month'] = RawData['date'].dt.month.astype('int')
RawData['Year'] = RawData['date'].dt.year.astype('int')

## Add Season

In [9]:
#Add Season, which starts after July and ends before July the following year

conditions = [
    RawData['Year'] == 2019,
    (RawData['Year'] == 2020) & (RawData['Month'] < 7),
    (RawData['Year'] == 2020) & (RawData['Month'] >= 7),
    (RawData['Year'] == 2021) & (RawData['Month'] < 7),
    (RawData['Year'] == 2021) & (RawData['Month'] >= 7),
    (RawData['Year'] == 2022) & (RawData['Month'] < 7),
    (RawData['Year'] == 2022) & (RawData['Month'] >= 7),
    (RawData['Year'] == 2023) & (RawData['Month'] < 7),
    (RawData['Year'] == 2023) & (RawData['Month'] >= 7),
    (RawData['Year'] == 2024) & (RawData['Month'] < 7),
    (RawData['Year'] == 2024) & (RawData['Month'] >= 7)
]

choices = [
    '2019/2020',
    '2019/2020',
    '2020/2021',
    '2020/2021',
    '2021/2022',
    '2021/2022',
    '2022/2023',
    '2022/2023',
    '2023/2024',
    '2023/2024',
    '2024/2025'
]

RawData['Season'] = np.select(conditions, choices, default=np.nan)

## Add Quarters

In [10]:
#Add Quarter of the year as column, extracted from the month column
RawData['quarter'] = np.where((RawData['Month'] >= 1) & (RawData['Month'] <= 3), 1,
                             np.where((RawData['Month'] >= 4) & (RawData['Month'] <= 6), 2,
                                      np.where((RawData['Month'] >= 7) & (RawData['Month'] <= 9), 3,
                                               np.where((RawData['Month'] >= 10) & (RawData['Month'] <= 12), 4, np.nan))))

## Time Handling

In [11]:
RawData['Time'] = RawData['Time'].replace("Unknown", None)
RawData['Time'] = pd.to_datetime(RawData['Time'], format='%I:%M %p').dt.hour

## Drop Covid

Because of the Covid Pandemic and restrictions we decided to remove certain rows from our dataset. Due to the reason that during the time of Covid the Attendance was significantly influenced by Predictions, which will influence / worsen our predictions

 source drop of restriction: https://www.sfl-org.ch/fr/medias/communiques-officiels/article/la-saison-202122-peut-debuter-sans-restriction/

 source beginning of restrictions: https://en.wikipedia.org/wiki/2019%E2%80%9320_Swiss_Super_League?utm_source=chatgpt.com


In [12]:
# Months of Covid
months_to_remove_2020 = [3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
months_to_remove_2021 = [1, 2, 3, 4, 5]

# Remove rows for the period from March 2020 to May 2021
RawData = RawData[~(((RawData["Year"] == 2020) & (RawData["Month"].isin(months_to_remove_2020))) |
          ((RawData["Year"] == 2021) & (RawData["Month"].isin(months_to_remove_2021))))]

## Adding Stadium and Location Informations

In [13]:
# Define a dictionary containing stadium information for each team
stadium_info = {
    'FC Zürich': ['Letzigrund', 26104, 'Zürich', 'ZH', 1],
    'Grasshoppers': ['Letzigrund', 26104, 'Zürich', 'ZH', 1],
    'FC Lugano': ['Cornaredo Stadium', 6330, 'Lugano', 'TI', 0],
    'FC Luzern': ['Swissporarena', 16800, 'Lucerne', 'LU', 1],
    'FC Sion': ['Stade Tourbillon', 16232, 'Sion', 'VS', 1],
    'Servette FC': ['Stade de Genève', 30084, 'Lancy', 'GE', 1],
    'BSC Young Boys': ['Wankdorf Stadium', 31783, 'Berne', 'BE', 1],
    'FC St. Gallen': ['kybunpark', 20029, 'St. Gallen', 'SG', 1],
    'FC Basel': ['St. Jakob-Park', 38512, 'Basel', 'BS', 1],
    'FC Winterthur': ['Stadion Schützenwiese', 8550, 'Winterthur', 'ZH', 0],
    'Lausanne-Sport': ['Stade de la Tuilière', 12544, 'Lausanne', 'VD', 1],
    'Yverdon Sport': ['Stade Municipal', 6600, 'Yverdon', 'VD', 0]
}

# Iterate through the dictionary and update the DataFrame accordingly
for team, stadium_data in stadium_info.items():
    RawData.loc[RawData['Home Team'] == team,
                           ['Stadium', 'Max Capacity', 'City', 'Canton', 'Full Roof']] = stadium_data



## Remove Games not in Swiss Super League Stadium

In [14]:
# List of teams to keep
teams_to_keep = [
    'FC Zürich', 'Grasshoppers', 'FC Lugano', 'FC Luzern',
    'FC Sion', 'Servette FC', 'BSC Young Boys',
    'FC St. Gallen', 'FC Basel', 'FC Winterthur',
    'Lausanne-Sport', 'Yverdon Sport'
]

# Keep only rows where the "Home Team" is in the list of specified teams
RawData = RawData[RawData["Home Team"].isin(teams_to_keep)]

## Remove Games not Yet played

In [15]:
# Remove rows where the "Result" column has the value "-:-"
RawData = RawData[RawData["Result"] != "-:-"]

 ## Handling Stadium Attendance

In [16]:
#transform Attendance Format (e.g. 9.444 into 9444)
RawData['Attendance'] = RawData['Attendance'].astype(str).str.replace('.', '')

In [17]:
RawData = RawData[(RawData["Attendance"] != "x") & (RawData["Attendance"] != "nan")]

## Add Attendance in Percentage

In [18]:
# Convert 'Attendance' and 'Max Capacity' columns to numeric, handling any errors
RawData['Attendance'] = pd.to_numeric(RawData['Attendance'], errors='coerce')
RawData['Max Capacity'] = pd.to_numeric(RawData['Max Capacity'], errors='coerce')

# Remove rows with NaN values in 'Attendance' or 'Max Capacity' after conversion
RawData.dropna(subset=['Attendance', 'Max Capacity'], inplace=True)

# Calculate the 'PercentageAttendance' column
RawData['PercentageAttendance'] = RawData['Attendance'] / RawData['Max Capacity']

In [19]:
count = RawData[RawData['PercentageAttendance'] == 1].shape[0]
print(f"Number of occurrences where PercentageAttendecance is 1: {count}")

Number of occurrences where PercentageAttendecance is 1: 5


In [20]:
count = RawData[RawData['PercentageAttendance'] > 1].shape[0]
print(f"Number of occurrences where PercentageAttendecance is over 1: {count}")
# Standing Areas: Some stadiums have areas where people can stand rather than sit, allowing them to accommodate more people than their official seating capacity.
# Temporary Seating: During special events or high-profile games, temporary seating may be added to increase capacity.

Number of occurrences where PercentageAttendecance is over 1: 17


### Handling Rankings

In [21]:
RawData["Ranking Home Team"] = RawData["Ranking Home Team"].str.replace(r"[().]", "", regex=True).str.strip()
RawData["Ranking Away Team"] = RawData["Ranking Away Team"].str.replace(r"[().]", "", regex=True).str.strip()

## Remove Doubles

In [22]:
# Create a unique match identifier by including the competition, season, sorted home/away teams, and month
RawData['Match_ID'] = RawData.apply(lambda row: f"{row['Competition']}-{row['Season']}-{row['Month']}-{'-'.join(sorted([row['Home Team'], row['Away Team']]))}", axis=1)

# Drop duplicate games based on the generated 'Match_ID'
RawData = RawData.drop_duplicates(subset='Match_ID')

# Drop the 'Match_ID' column if it's no longer needed
RawData = RawData.drop(columns=['Match_ID'])

## Add Holidays

In [23]:
#Add holiday
SwissHoliday = pd.read_csv(path + "SwissHoliday.csv")
SwissHoliday

,Date,BE,VD,TI,VS,LU,ZH,BS,SG,GE
0,01-01-2019,1,1,1,1,1,1,1,1,1
1,01-02-2019,1,1,0,0,0,0,0,0,0
2,01-06-2019,0,0,1,0,0,0,0,0,0
3,01-06-2019,0,0,0,0,0,0,0,0,0
4,03-01-2019,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
175,12-08-2024,0,0,1,1,1,0,0,0,0
176,12-08-2024,0,0,0,0,0,0,0,0,0
177,12-25-2024,1,1,1,1,1,1,1,1,1
178,12-26-2024,1,0,1,0,1,1,1,1,0


In [24]:
# Define column names
date_col = 'Date'  # Assuming 'Date' is the name of your date column in both datasets
canton_col = 'Canton'  # Assuming 'Canton' is the name of the canton column in RawData
holiday_col = 'Holiday'  # Assuming 'Holiday' is the name of the new holiday column in RawData
swiss_cantons = SwissHoliday.columns[1:] # Assuming the first column is 'Date' and the rest are canton names

# Convert 'Date' columns to datetime objects if they are not already
RawData[date_col] = pd.to_datetime(RawData[date_col])
SwissHoliday[date_col] = pd.to_datetime(SwissHoliday[date_col])

# Create the 'holiday' column in RawData and initialize it to 0
RawData[holiday_col] = 0

# Iterate through RawData and check for matches in SwissHoliday
for index, row in RawData.iterrows():
    # Get the canton for this row
    canton = row[canton_col]

    # Check if this canton is in SwissHoliday columns
    if canton in swiss_cantons:
        # Check if the date is a holiday for this canton
        if SwissHoliday[(SwissHoliday[date_col] == row[date_col]) & (SwissHoliday[canton] == 1)].shape[0] > 0:
            RawData.loc[index, holiday_col] = 1

In [25]:
holiday_count = RawData['Holiday'].sum()
print(f"Total number of holidays: {holiday_count}")

Total number of holidays: 23


## Add Quartely GDP --> but lagged for Q3 in Stadium Added use Q2 in SwissGdp

In [26]:
#Import Swiss GDPs
SwissGDPS = pd.read_csv(path + "SwissGDP.csv")

new_gdp_data = [0.0360, 0.2793, 0.3926, 0.1423, 0.2062, 0.0708, 0.1486, 1.5024, 0.1414, 2024, 3]
SwissGDPS.loc[len(SwissGDPS)] = new_gdp_data

canton_cols = [col for col in SwissGDPS.columns if col not in ['Year', 'quarter']]  # Exclude 'Year' and 'quarter'
new_canton_cols = {col: f'Canton_{col}' for col in canton_cols}
SwissGDPS = SwissGDPS.rename(columns=new_canton_cols)

In [27]:
import pandas as pd

# Create dummy variables for cantons
canton_dummies = pd.get_dummies(RawData['Canton'], prefix='Canton')

# Concatenate the dummy variables with the original DataFrame
RawData = pd.concat([RawData, canton_dummies], axis=1)



In [28]:
import pandas as pd

# Ensure 'Year' and 'quarter' are integers for proper comparison
RawData['Year'] = RawData['Year'].astype(int)
RawData['quarter'] = RawData['quarter'].astype(int)
# ... (Your existing code to load and process RawData and SwissGDPS) ...

# Ensure 'Year' and 'quarter' are integers for proper comparison
RawData['Year'] = RawData['Year'].astype(int)
RawData['quarter'] = RawData['quarter'].astype(int)

# Check if 'Year' and 'quarter' columns exist in SwissGDPS
# If they exist, convert them to integers; otherwise, handle the case
if 'Year' in SwissGDPS.columns:
    SwissGDPS['Year'] = SwissGDPS['Year'].astype(int)
if 'quarter' in SwissGDPS.columns:
    SwissGDPS['quarter'] = SwissGDPS['quarter'].astype(int)
else:
    print("Warning: 'Year' or 'quarter' column not found in SwissGDPS. Please check your data.")



In [29]:
import pandas as pd

# Ensure 'Year' and 'quarter' are integers for proper comparison
RawData['Year'] = RawData['Year'].astype(int)
RawData['quarter'] = RawData['quarter'].astype(int)

# Check if 'Year' and 'quarter' columns still have their original names
# If they have been renamed, use the renamed column names
year_col = 'Year' if 'Year' in SwissGDPS.columns else SwissGDPS.columns[SwissGDPS.columns.str.contains('Year', case=False)].values[0]
quarter_col = 'quarter' if 'quarter' in SwissGDPS.columns else SwissGDPS.columns[SwissGDPS.columns.str.contains('quarter', case=False)].values[0]

# Convert 'Year' and 'quarter' columns to integers in SwissGDPS
SwissGDPS[year_col] = SwissGDPS[year_col].astype(int)
SwissGDPS[quarter_col] = SwissGDPS[quarter_col].astype(int)

# Add a new column 'Last Month GDP' to RawData and initialize to NaN
RawData['Last Month GDP'] = pd.NA

# Iterate through RawData and find matching lagged GDP in SwissGDPS
for index, row in RawData.iterrows():
    year = row['Year']
    quarter = row['quarter']
    canton = row['Canton']  # Get the canton from RawData

    # Calculate lagged quarter
    lagged_quarter = quarter - 1
    if lagged_quarter == 0:
        lagged_quarter = 4
        year -= 1  # Adjust year for Q1 of the previous year

    # Find matching lagged GDP in SwissGDPS
    matching_gdp = SwissGDPS[
        (SwissGDPS[year_col] == year) &
        (SwissGDPS[quarter_col] == lagged_quarter)
    ]

    # Check if a match was found and assign GDP value
    if not matching_gdp.empty:
        # Access the GDP value for the specific canton
        gdp_column = f"Canton_{canton}"
        if gdp_column in matching_gdp.columns:
            gdp_value = matching_gdp[gdp_column].values[0]
            RawData.loc[index, 'Last Month GDP'] = gdp_value



## Result Handling (1)

In [30]:
# Splitting the "Result" column into "Home Team Goals Scored" and "Away Team Goals Scored, some "score" rows contain - as they haven"t been played yet.
RawData[["Home Team Goals Scored", "Away Team Goals Scored"]] = RawData["Result"].apply(
    lambda x: (x.split(":") if ":" in x else ["-", "-"])
).apply(pd.Series)

In [31]:
RawData.drop(columns=["Result"], inplace=True)

In [32]:
# Creating a new column "Match Type" based on conditions
def determine_match_type(value):
    if "AET" in value:
        return "Extra Time"
    elif "pens" in value:
        return "Penalties"
    else:
        return "Normal Time"

# Applying the function to create the "Match Type" column
RawData["Match Type"] = RawData["Away Team Goals Scored"].apply(determine_match_type)

# Optionally, remove any letters like "AET" or "on pens" from the score columns to keep only the numbers
RawData["Away Team Goals Scored"] = RawData["Away Team Goals Scored"].str.extract(r"(\d+)").astype(int)

## Add Win/Lost

In [33]:
# Ensure the goal columns are numeric and handle non-numeric entries (e.g., "-")
RawData["Home Team Goals Scored"] = pd.to_numeric(RawData["Home Team Goals Scored"], errors="coerce")
RawData["Away Team Goals Scored"] = pd.to_numeric(RawData["Away Team Goals Scored"], errors="coerce")

# Function to determine the match result for home and away teams
def determine_outcome(home_goals, away_goals):
    if pd.isna(home_goals) or pd.isna(away_goals):
        return "Not Played", "Not Played"  # Assign "Not Played" if either score is missing
    elif home_goals > away_goals:
        return "Win", "Loss"
    elif home_goals < away_goals:
        return "Loss", "Win"
    else:
        return "Draw", "Draw"

# Applying the function to create "Home Team Outcome" and "Away Team Outcome" columns
RawData[["Home Team Outcome", "Away Team Outcome"]] = RawData.apply(
    lambda row: pd.Series(determine_outcome(row["Home Team Goals Scored"], row["Away Team Goals Scored"])),
    axis=1
)

## Result Handling (2) (Manual)

In [34]:
# Match 1: Yverdon Sport vs. FC Zürich (26 October 2021)
target_row = RawData[(RawData["Weekday"] == "Tuesday") & (RawData["Month"] == 10) & (RawData["Year"] == 2021) &
                (RawData["Home Team"] == "Yverdon Sport") & (RawData["Away Team"] == "FC Zürich")]
if not target_row.empty:
    index = target_row.index[0]
    RawData.loc[index, "Home Team Goals Scored"] = 2
    RawData.loc[index, "Away Team Goals Scored"] = 2

# Match 2: Servette FC vs. FC Lugano (5 April 2023)
target_row = RawData[(RawData["Weekday"] == "Wednesday") & (RawData["Month"] == 4) & (RawData["Year"] == 2023) &
                (RawData["Home Team"] == "Servette FC") & (RawData["Away Team"] == "FC Lugano")]
if not target_row.empty:
    index = target_row.index[0]
    RawData.loc[index, "Home Team Goals Scored"] = 2
    RawData.loc[index, "Away Team Goals Scored"] = 2

# Match 3: Servette FC vs. Lausanne-Sport (1 November 2023)
target_row = RawData[(RawData["Weekday"] == "Wednesday") & (RawData["Month"] == 11) & (RawData["Year"] == 2023) &
                (RawData["Home Team"] == "Servette FC") & (RawData["Away Team"] == "Lausanne-Sport")]
if not target_row.empty:
    index = target_row.index[0]
    RawData.loc[index, "Home Team Goals Scored"] = 1
    RawData.loc[index, "Away Team Goals Scored"] = 0

# Match 4: Servette FC vs. FC Lugano (2 June 2024)
target_row = RawData[(RawData["Weekday"] == "Sunday") & (RawData["Month"] == 6) & (RawData["Year"] == 2024) &
                (RawData["Home Team"] == "Servette FC") & (RawData["Away Team"] == "FC Lugano")]
if not target_row.empty:
    index = target_row.index[0]
    RawData.loc[index, "Home Team Goals Scored"] = 0
    RawData.loc[index, "Away Team Goals Scored"] = 0

# Match 5: FC Lugano vs. FC Luzern (21 April 2022)
target_row = RawData[(RawData["Weekday"] == "Thursday") & (RawData["Month"] == 4) & (RawData["Year"] == 2022) &
                (RawData["Home Team"] == "FC Lugano") & (RawData["Away Team"] == "FC Luzern")]
if not target_row.empty:
    index = target_row.index[0]
    RawData.loc[index, "Home Team Goals Scored"] = 2
    RawData.loc[index, "Away Team Goals Scored"] = 2

# Match 6: FC Basel vs. FC Lugano (28 February 2024)
target_row = RawData[(RawData["Weekday"] == "Wednesday") & (RawData["Month"] == 2) & (RawData["Year"] == 2024) &
                (RawData["Home Team"] == "FC Basel") & (RawData["Away Team"] == "FC Lugano")]
if not target_row.empty:
    index = target_row.index[0]
    RawData.loc[index, "Home Team Goals Scored"] = 1
    RawData.loc[index, "Away Team Goals Scored"] = 1

# Match 7: FC Basel vs. Bröndby IF (11 August 2022)
target_row = RawData[(RawData["Weekday"] == "Thursday") & (RawData["Month"] == 8) & (RawData["Year"] == 2022) &
                (RawData["Home Team"] == "FC Basel") & (RawData["Away Team"] == "Bröndby IF")]
if not target_row.empty:
    index = target_row.index[0]
    RawData.loc[index, "Home Team Goals Scored"] = 2
    RawData.loc[index, "Away Team Goals Scored"] = 1

## Add Wins/Goals Scored/Goals conceded

In [35]:
# Parse the 'Date' column to extract Year, Month, and Day
RawData['Date'] = pd.to_datetime(RawData['Date'], format='%m-%d-%Y', errors='coerce')  # Convert to datetime format, handle multiple formats

# Extract Year, Month, and Day
RawData['Year'] = RawData['Date'].dt.year
RawData['Month'] = RawData['Date'].dt.month
RawData['Day'] = RawData['Date'].dt.day

# Sort by Year, Month, and Day
RawData = RawData.sort_values(by=['Year', 'Month', 'Day']).reset_index(drop=True)

# Create a temporary column to track wins for rolling calculations
RawData['Win'] = (RawData['Home Team Outcome'] == 'Win').astype(int)

# Rolling calculations for the last 5 games for each team
RawData['Goals Scored in Last 5 Games'] = RawData.groupby('Home Team')['Home Team Goals Scored'].rolling(window=5, min_periods=1).sum().shift(1).reset_index(level=0, drop=True)
RawData['Goals Conceded in Last 5 Games'] = RawData.groupby('Home Team')['Away Team Goals Scored'].rolling(window=5, min_periods=1).sum().shift(1).reset_index(level=0, drop=True)
RawData['Number of Wins in Last 5 Games'] = RawData.groupby('Home Team')['Win'].rolling(window=5, min_periods=1).sum().shift(1).reset_index(level=0, drop=True)

# Manually correct the first 5 games for each team using provided historical data
historical_data = {
    'FC Zürich': {'Goals Scored': [1, 0, 3, 1, 0], 'Goals Conceded': [1, 3, 0, 1, 1], 'Wins': [0, 0, 1, 0, 0]},
    'Grasshopper Club Zürich': {'Goals Scored': [3, 0, 0, 0, 1], 'Goals Conceded': [3, 3, 4, 6, 3], 'Wins': [0, 0, 0, 0, 0]},
    'FC Lugano': {'Goals Scored': [3, 1, 1, 1, 0], 'Goals Conceded': [3, 0, 0, 3, 0], 'Wins': [0, 1, 1, 0, 0]},
    'FC Luzern': {'Goals Scored': [0, 3, 4, 0, 0], 'Goals Conceded': [1, 0, 0, 1, 0], 'Wins': [0, 1, 1, 0, 0]},
    'FC Sion': {'Goals Scored': [0, 3, 0, 1, 0], 'Goals Conceded': [4, 0, 1, 1, 1], 'Wins': [0, 1, 0, 1, 0]},
    'Servette FC': {'Goals Scored': [0, 2, 3, 1, 5], 'Goals Conceded': [1, 5, 1, 1, 1], 'Wins': [0, 0, 1, 0, 1]},
    'BSC Young Boys': {'Goals Scored': [4, 6, 1, 1, 2], 'Goals Conceded': [0, 1, 0, 0, 0], 'Wins': [1, 1, 1, 1, 1]},
    'FC St. Gallen': {'Goals Scored': [1, 2, 1, 0, 0], 'Goals Conceded': [1, 3, 0, 1, 2], 'Wins': [0, 0, 1, 0, 0]},
    'FC Basel': {'Goals Scored': [4, 3, 1, 0, 3], 'Goals Conceded': [1, 2, 0, 1, 1], 'Wins': [1, 1, 1, 0, 1]},
    'FC Winterthur': {'Goals Scored': [1, 1, 1, 1, 1], 'Goals Conceded': [1, 1, 1, 1, 1], 'Wins': [0, 0, 0, 0, 0]},
    'Lausanne-Sport': {'Goals Scored': [1, 1, 1, 1, 1], 'Goals Conceded': [1, 1, 1, 1, 1], 'Wins': [0, 0, 0, 0, 0]},
    'Yverdon Sport': {'Goals Scored': [1, 2, 2, 1, 3], 'Goals Conceded': [1, 1, 0, 1, 0], 'Wins': [0, 1, 1, 0, 1]},
}

for team, data in historical_data.items():
    mask = (RawData['Home Team'] == team) & (RawData.index < 5)
    RawData.loc[mask, 'Goals Scored in Last 5 Games'] = sum(data['Goals Scored'])
    RawData.loc[mask, 'Goals Conceded in Last 5 Games'] = sum(data['Goals Conceded'])
    RawData.loc[mask, 'Number of Wins in Last 5 Games'] = sum(data['Wins'])

# Replace NaT and NaN values with 0 for the first few rows where there isn't enough data
RawData.fillna(0, inplace=True)

# Drop the temporary 'Win' column as it's no longer needed
RawData.drop(columns=['Win'], inplace=True)

<ipython-input-35-4c9ee9bf909e>:43: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  RawData.fillna(0, inplace=True)


## Drop date and old date

In [36]:
RawData = RawData.drop(columns=['old_date','date'])

## Categorization of some columns

In the following, certain Features will be categorized (Ranking, Weather, Time, Competition, Day of the Week, Derby). This is done to make it easier for our models to capture the importance of different features

In [37]:
RawData['Ranking Home Team'] = pd.to_numeric(RawData['Ranking Home Team'], errors='coerce').fillna(0).astype(int)
RawData['Ranking Away Team'] = pd.to_numeric(RawData['Ranking Away Team'], errors='coerce').fillna(0).astype(int)


In [38]:
#Categorize The ranking into different categories
def categorize_opposing_team(ranking):
    if ranking in range(1, 4):
        return "Top ranked"
    elif ranking in range(4, 9):
        return "Medium ranked"
    elif ranking in range(9, 13):
        return "Bottom ranked"
    elif ranking == 0:
        return "Not ranked"
    else:
        return "Unknown"  # Just in case of unexpected values

RawData['Opposing team Category'] = RawData['Ranking Away Team'].apply(categorize_opposing_team)
RawData['Home team Category'] = RawData['Ranking Home Team'].apply(categorize_opposing_team)

In [39]:
def categorize_game_day(weekday):
    if weekday in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']:
        return "Weekday"
    elif weekday in ['Saturday', 'Sunday']:
        return "Weekend"
    else:
        return "Unknown"  # In case of unexpected values

# Applying the categorization function to create the new feature
RawData['Game day'] = RawData['Weekday'].apply(categorize_game_day)

In [40]:
# Ensure the Time column is in string format with "hh:mm"
RawData['Time'] = RawData['Time'].astype(str)

# Creating the new feature 'Time slot' based on the existing feature 'Time'
def categorize_time_slot(time_str):
    try:
        # Extract hour from the time string (assumes format "hh:mm")
        hour = int(time_str.split(':')[0])
        if hour < 18:
            return "Afternoon"
        elif 18 <= hour < 20:
            return "Evening"
        else:
            return "Night"
    except:
        return "Unknown"  # In case of unexpected or incorrect values

# Applying the categorization function to create the new feature
RawData['Time slot'] = RawData['Time'].apply(categorize_time_slot)

# Display the updated DataFrame
print(RawData.head())

        Competition Matchday  Time       Home Team  Ranking Home Team  \
0      Super League        1  20.0         FC Sion                  8   
1      Super League        1  19.0   FC St. Gallen                  2   
2  Challenge League        1  17.0   FC Winterthur                  4   
3      Super League        1  16.0       FC Zürich                  7   
4      Super League        1  16.0  BSC Young Boys                  1   

     Away Team  Ranking Away Team  Attendance                Weather  \
0     FC Basel                  3       11000  Clear or mostly clear   
1    FC Luzern                  6       11672          Partly cloudy   
2     FC Aarau                  8        3800          Partly cloudy   
3    FC Lugano                  5        9430  Clear or mostly clear   
4  Servette FC                  4       25110  Clear or mostly clear   

   Temperature (°C)  ... Home Team Outcome Away Team Outcome  Day  \
0              24.6  ...              Loss               Wi

In [41]:
unique_weather_categories = RawData['Weather'].unique()

# Display the unique weather categories
print(unique_weather_categories)

['Clear or mostly clear' 'Partly cloudy' 'Rainy' 'Drizzle' 'Snowy' 0]


In [42]:
def categorize_weather(weather):
    if weather in ['Clear or mostly clear', 'Partly cloudy']:
        return "Good"
    elif weather in ['Rainy', 'Drizzle', 'Snowy']:
        return "Bad"
    else:
        return "Unknown"  # In case of unexpected or incorrect values

# Applying the categorization function to create the new feature
RawData['Weather GoodBad'] = RawData['Weather'].apply(categorize_weather)


In [43]:
def is_derby_boolean(home_team, away_team):
    # Defining all the derbies
    derbies = {
        'FC Zürich': ['Grasshopper Club Zürich'],
        'Grasshopper Club Zürich': ['FC Zürich'],
        'BSC Young Boys': ['FC Thun', 'FC Basel'],
        'FC Basel': ['Grasshopper Club Zürich', 'BSC Young Boys', 'FC Zürich'],
        'FC Luzern': ['SC Kriens'],
        'FC St. Gallen': ['FC Wil', 'FC Luzern'],
        'Servette FC': ['FC Lausanne-Sport', 'FC Sion'],
        'FC Lausanne-Sport': ['Servette FC', 'Yverdon Sport FC'],
        'FC Sion': ['Servette FC', 'FC Lausanne-Sport'],
        'Yverdon Sport FC': ['FC Lausanne-Sport'],
        'FC Winterthur': ['Grasshopper Club Zürich', 'FC Zürich'],
        'FC Lugano': ['AC Bellinzona', 'FC Chiasso']
    }

    # Check if the home and away team form a derby
    if away_team in derbies.get(home_team, []):
        return 1
    elif home_team in derbies.get(away_team, []):
        return 1
    else:
        return 0

# Applying the categorization function to create the new feature
RawData['Derby'] = RawData.apply(lambda row: is_derby_boolean(row['Home Team'], row['Away Team']), axis=1)

In [44]:
# Manual Checking of dataset
# Save the DataFrame to a CSV file
output_csv = "football_results.csv"
RawData.to_csv(output_csv, index=False)

from google.colab import files

# Download the CSV file
# This CSV File is then further used in the notebook for the Models
files.download(output_csv)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>